In [1]:
import io
import re
from main import initial_path
DIR_PATH = initial_path()
import os
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from utils.text_analysis import TextAnalysis
from runs.rss_scraping import rss_scrap, urls
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import nltk
nltk.download('cess_esp')
words = set( nltk.corpus.cess_esp.words())

[nltk_data] Downloading package cess_esp to C:\Users\Adrian
[nltk_data]     Bolanos\AppData\Roaming\nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


In [2]:
ta = TextAnalysis('es')

Language: es
Text Analysis: ['emoji', 'tagger', 'parser', 'stemmer', 'ner']


In [3]:
data_raw = pd.read_csv(DIR_PATH, sep=';')
data_raw.head()

,title,content,type
0,Los elefantes que pensó Alexánder Robledo,Dice que le gustaría tener un contacto real co...,Cultura
1,Otra oportunidad para ver lo mejor de 2016,"- 17 años - Jericó, el infinito vuelo de los d...",Cultura
2,SENSACIÓN CREPUSCULAR,espera el milagroso esquila del rebaño venda...,Cultura
3,"Salen los jugadores, entran los directivos",La temporada que recién terminó para Nacional ...,Deportes
4,"Como sexta en el ranking de la Fifa, Colombia ...",11. Suiza 1129 Sudamérica conserva su ...,Deportes


In [4]:
setting = {'url': False, 'mention': False, 'emoji': False, 'hashtag': False, 'stopwords': True, 'relabel': False} 
list_sentences = []
content_list =  data_raw['content'].to_list()
for row in tqdm(content_list):
    text = ta.clean_text(row, **setting)
    #Eliminando direcciones de correo
    text = re.sub('\S*@\S*\s?','',text)
    #Eliminando palabras en ingles
    text = " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha())
    list_sentences.append(text)

100%|██████████| 1635/1635 [01:04<00:00, 25.41it/s]


In [5]:
print('Text org: {0} \n\nTex clean: {1}'.format(content_list[1], list_sentences[1]))

Text org: - 17 años - Jericó, el infinito vuelo de los días Dos veces al año el Colombo Americano y la Revista Kinetoscopio hacen el recuento de lo Mejor del año, una selección que escoge una serie de películas que presentaron en cada semestre y que para el criterio sus expertos son lo más destacado. Para la selección, que la hace el departamento de cine de la Revista Kinetoscopio, tienen en cuenta la asistencia a las salas durante ese periodo, además de los reconocimientos y la participación en festivales internacionales.  - Julieta - Aislados Toda la programación tiene boletería 2x1. - Camino a Estambul - La chica del tren 
			Prohibida su reproducción total o parcial. La traducción a cualquier idioma está permitida estrictamente para usos pedagógicos y debe citarse la fuente. Reproduction in whole or in part is forbidden. Translation in any language is permitted strictly for pedagogic uses without written permission, and credits shoud be given to EL MUNDO.
      Webmaster: webmaster

In [6]:
x = list_sentences
y = data_raw['type'].to_list()

In [7]:
print('**Replica y_train:', sorted(Counter(y).items()))

**Replica y_train: [('Ciencia y Tecnologia', 327), ('Cultura', 324), ('Deportes', 334), ('Economia', 330), ('Salud', 320)]


In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=8675309)
print('**Replica train: {0}, size {1}'.format(sorted(Counter(y_train).items()), len(y_train)))
print('**Replica test: {0}, size {1}'.format(sorted(Counter(y_test).items()), len(y_test)))

**Replica train: [('Ciencia y Tecnologia', 230), ('Cultura', 227), ('Deportes', 252), ('Economia', 224), ('Salud', 211)], size 1144
**Replica test: [('Ciencia y Tecnologia', 97), ('Cultura', 97), ('Deportes', 82), ('Economia', 106), ('Salud', 109)], size 491


## Features in Bag of Words

In [9]:
vec = CountVectorizer(min_df=5, ngram_range=(1,3), max_features=5000, strip_accents='unicode', lowercase =True, analyzer='word')
vec.fit(x_train)
x_train = vec.transform(x_train)
x_test = vec.transform(x_test)

In [10]:
 print(vec.get_feature_names())

['abajo', 'abierta', 'abiertas', 'abierto', 'abiertos', 'abordar', 'abre', 'abre puerta', 'abren', 'abril', 'abrio', 'abrir', 'abundancia', 'abuso', 'acaba', 'acaban', 'acabar', 'acabo', 'academico', 'acceder', 'acceso', 'accidente', 'accidentes', 'accion', 'acciones', 'acelerar', 'aceptacion', 'aceptar', 'acerca', 'acero', 'acompanado', 'aconseja', 'acoso', 'actitudes', 'activa', 'activacion', 'activar', 'activas', 'actividad', 'actividad economica', 'actividad fisica', 'actividades', 'activo', 'activos', 'acto', 'actor', 'actores', 'actuacion', 'actuaciones', 'actuaciones asegurado', 'actuaciones asegurado comunicando', 'actual', 'actuales', 'actualidad', 'actualidad cientifica', 'actualidad cientifica favor', 'actuan', 'actuar', 'acude', 'acude servicio', 'acude servicio medico', 'acudir', 'acudir redaccion', 'acuerdos', 'acumula', 'acumulacion', 'adaptacion', 'adaptado', 'adaptar', 'adecuada', 'adecuadamente', 'adecuadas', 'adecuado', 'adecuados', 'adelanta', 'adelantado', 'adelant

In [11]:
print(x_train.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]


## Model

In [12]:
classifier = LogisticRegression(C=10, penalty='l1',solver='liblinear', multi_class='auto',max_iter=1000) 
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

## Cross Validation

In [13]:
cv_score = np.mean(cross_val_score(classifier, x_train, y_train, cv=5, scoring='accuracy'))
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: {}%'.format(round(accuracy, 2)*100))
print('Accuracy with CV: {}%'.format(round(cv_score, 2)*100))

Accuracy: 93.0%
Accuracy with CV: 92.0%


## Metrics

In [14]:
print(confusion_matrix(y_test, y_pred))
print("")
print(classification_report(y_test, y_pred))

[[ 88   4   0   1   4]
 [  2  84   3   5   3]
 [  0   2  80   0   0]
 [  0   2   1 103   0]
 [  2   4   0   0 103]]

                      precision    recall  f1-score   support

Ciencia y Tecnologia       0.96      0.91      0.93        97
             Cultura       0.88      0.87      0.87        97
            Deportes       0.95      0.98      0.96        82
            Economia       0.94      0.97      0.96       106
               Salud       0.94      0.94      0.94       109

            accuracy                           0.93       491
           macro avg       0.93      0.93      0.93       491
        weighted avg       0.93      0.93      0.93       491



In [15]:
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='macro')
precision = precision_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print('Accuracy: {}%'.format(round(accuracy, 2)*100))
print('Recall: {}%'.format(round(recall, 2)*100))
print('Precision: {}%'.format(round(precision, 2)*100))
print('F1: {}%'.format(round(f1, 2)*100))

Accuracy: 93.0%
Recall: 93.0%
Precision: 93.0%
F1: 93.0%
